In [1]:
#flashcard class
class flashcard():
    def __init__(self, qns, ans, n_correct=0, n_wrong=0, review_time=0):
        self._qns = qns
        self._ans = ans 
        self._n_correct = n_correct
        self._n_wrong = n_wrong
        self._review_time = review_time
        
    def get_qns(self):
        return self._qns
    def get_ans(self):
        return self._ans
    def get_n_correct(self):
        return self._n_correct
    def get_n_wrong(self):
        return self._n_wrong
    def get_review_time(self):
        return self._review_time
    
    def set_qns(self,qns):
        self._qns=qns
    def set_ans(self,ans):
        self._ans=ans
    def set_n_correct(self,n_correct):
        self._n_correct=n_correct
    def set_n_wrong(self,n_wrong):
        self._n_wrong=n_wrong
    def set_review_time(self,review_time):
        self._review_time=review_time

        
    def __str__(self):
        ret= f"{self._qns}, {self._ans}, {self._n_correct}, {self._n_wrong}, {self._review_time}"
        return ret
    
    def __lt__(self, other):
        #Criteria for prioritizing flashcards based on the length of the question
        return len(self._qns) < len(other._qns)
    


In [2]:

####
# This code contains the algorithm to obtain samples from optimal reviewing intensity.
# This script read two csv files, preprocesed_weights.csv, observations_1k.csv and outputs revieiw times.
# For user-item pairs that are not reviewed in the period, we show empty string.
####

import numpy as np

Q = 1.0  # parameter Q defined in eq. 8 in the paper. where Q is a given parameter, which trades off recall probability and number of item reviews—the higher its value, the lower the number of reviews.
T = 10.0  # number of days in the future to generate reviewing timeself.


#this gets the intensity to be used in the sampler function
def intensity(t, n_t, q):
    return 1.0/np.sqrt(q)*(1-np.exp(-n_t*t))

#apparently this function samples from an inhomogeneous Poisson process with intensity and it returns the sampled time
def sampler(n_t, q, T):
    t = 0
    while(True):
        max_int = 1.0/np.sqrt(q)
        t_ = np.random.exponential(1 / max_int)

        if t_ + t > T:
            return None
        t = t+t_
        proposed_int = intensity(t, n_t, q)
        if np.random.uniform(0, 1, 1)[0] < proposed_int / max_int:
            return t

#main function for calculating review time        
def calculate_review_time(flashcard,pqueue):
    
    #n_correct and n_wrong are for each indiv flashcard
    n_correct = flashcard.get_n_correct()
    n_wrong = flashcard.get_n_wrong()
    
    #correct and wrong are for the whole deck average
    correct=float(0)
    wrong=float(0)

    qsize=0
    while not pqueue.empty():
        item=pqueue.dequeue()
        qsize+=1
        correct+=int(item.get_n_correct())
        wrong+=int(item.get_n_wrong())
    
    if qsize!=0:
        correct=correct/qsize
        wrong=wrong/qsize

    n_t= 2**(-(correct*n_correct+wrong*n_wrong))
    t_rev = sampler(n_t, Q, T)
    return t_rev
        


In [5]:
flashcard1=flashcard('What is force?',"Force is the product of mass and acceleration.")
flashcard2=flashcard('What is magnetic flux linkage?','The magnetic flux linkage is defined as the product of the number of turns of the coil and the magnetic flux through each turn.')
flashcard3=flashcard('Define threshold frequency','The threshold frequency is the minimum frequency of the electromagnetic radaition required to liberate photoelectrons from a metal surface.')
flashcard4=flashcard('What is a work function?','The work function is the minimum amount of energy required to remove an electron from the metal surface.')
flashcard5=flashcard('How do you tell if a current is going into or out of a plane?','A dot represents the current coming out of the plane while a cross represents the current going into the plane.')
flashcard6=flashcard('Define e.m.f.','The e.m.f. of a source is defined as the energy converted per unit electric charge to electrical energy in driving charge round a complete circuit.')
flashcard7=flashcard('What is an ideal gas?','An ideal gas is a hypothetical gas that obeys the equation of state of an ideal gas for all pressures, volumes, number of moles and temperatures.')

import queue
from queue import PriorityQueue

#priorityqueue class uses imported PriorityQueue, but i customised it to make it easier to use
class pqueue():
    def __init__(self):
        self._queue = PriorityQueue()

#enqueue ranks by review time first, then by length of the question
    def enqueue(self, priority, item):
        self._queue.put((priority, item))

    def dequeue(self):
        if self._queue.empty():
            return None
        return self._queue.get()[1]
    
    def count(self):
        count=0
        for item in self._queue.queue:
            count+=1
        return count   
    
    def empty(self):
        count=0
        for item in self._queue.queue:
            count+=1
        if count==0:
            return True
        else:
            return False
    
    def __str__(self):
        items = []
        for item in self._queue.queue:
            items.append(item)
        return str(items)
    
p=pqueue()
print(flashcard1.get_review_time())
print(flashcard1)

p.enqueue(2, flashcard1)
p.enqueue(0.3, flashcard2)
p.enqueue(0.1, flashcard3)
p.enqueue(5, flashcard4)
p.enqueue(0.7, flashcard5)
p.enqueue(9, flashcard6)
p.enqueue(1, flashcard7)

for i in range(p.count()):
    print(p.dequeue())



0
What is force?, Force is the product of mass and acceleration., 0, 0, 0
Define threshold frequency, The threshold frequency is the minimum frequency of the electromagnetic radaition required to liberate photoelectrons from a metal surface., 0, 0, 0
What is magnetic flux linkage?, The magnetic flux linkage is defined as the product of the number of turns of the coil and the magnetic flux through each turn., 0, 0, 0
How do you tell if a current is going into or out of a plane?, A dot represents the current coming out of the plane while a cross represents the current going into the plane., 0, 0, 0
What is an ideal gas?, An ideal gas is a hypothetical gas that obeys the equation of state of an ideal gas for all pressures, volumes, number of moles and temperatures., 0, 0, 0
What is force?, Force is the product of mass and acceleration., 0, 0, 0
What is a work function?, The work function is the minimum amount of energy required to remove an electron from the metal surface., 0, 0, 0
Define

In [ ]:
p=PriorityQueue()
p.put((2.5,flashcard1))
p.put((0.3,flashcard2))
p.put((0.2,flashcard3))
p.put((4.1,flashcard4))

while p !=None:
    print(p.get())

(0.2, <__main__.flashcard object at 0x109916d70>)
(0.3, <__main__.flashcard object at 0x109915a80>)
(2.5, <__main__.flashcard object at 0x1099174c0>)
(4.1, <__main__.flashcard object at 0x1097fed40>)


In [4]:
def show_flashcard(pqueue):
    status=''
    while status != 'stop':
        flashcard_to_show=pqueue.dequeue()
        print(flashcard_to_show)
        print(pqueue.count())
        print(f'{flashcard_to_show.get_qns()}')
        status = input('Type C (correct) or W (wrong), or type "stop" to end: ').lower()
        if status == 'stop':
            break
        elif status == 'c':
            flashcard_to_show.set_n_correct(flashcard_to_show.get_n_correct() + 1)
        elif status == 'w':
            flashcard_to_show.set_n_wrong(flashcard_to_show.get_n_wrong() + 1)
        
        #attain new review time
        review_time = calculate_review_time(flashcard_to_show,pqueue)
        
        # Enqueue the modified flashcard back into the priority queue in new position
        pqueue.enqueue(review_time, flashcard_to_show)
    
show_flashcard(p)    
    

None
0


AttributeError: 'NoneType' object has no attribute 'get_qns'